Reference paper: stability is stable 
Experiment: vary the value of rho and see the minimum sample size required for replicability

func getConvergenceSampleNum(hyperparams)
    For sample_num range(min_subsets_size, max_subsets_size, step):
        For range(repeat_num):
            random draw a subset of the dataset, whose size = sample_num
            Use algorithm 10 to get a model
        Check whether the models we get are 'replicable' to each other, accoding to the hyperparams. (see definition in page 14 of the paper)
        if converged, return current sample_num

func experiment(): 
# vary the rho and see minumum sample size required for replicability. 
# To make it simple, we can fix the value of other hyperparamers, such as alpha and beta.
    for rho in range(min_rho, max_rho, step):
        sample_num = getConvergenceSampleNum(rho)
        theoretical_sample_num = getTheoreticalSampleNum(rho) # compute according to algorithm 10 in the paper
        print("Hyperparams: ", hyperparams, "Sample size: ", sample_num)
    plot(hyperparams, theoretical_sample_num) # draw the curve, where x-axis is the hyperparams and y-axis is the sample size
    plot(hyperparams, sample_num) # draw the curve, where x-axis is the hyperparams and y-axis is the sample size

In [1]:
import pandas as pd
import numpy as np
import config
import Algorithm10 as a10


##### Config Variables #####
np = <module 'numpy' from '/opt/anaconda3/envs/learn/lib/python3.12/site-packages/numpy/__init__.py'>
dataset_path = ./dataset/Invistico_Airline.csv
model_path = ./models/
max_depth = 3
random_seed = 42
selected_features = ['Class', 'Seat comfort', 'Food and drink', 'Cleanliness', 'satisfaction']
rho = 0.1
alpha = 0.3
beta = 0.01
num_H = 20
m = 100
m_up_bound = 2301155.406349028
tau_up_bound = 0.010014246020860021
tau = 1.0014246020860022e-06
############################


In [3]:
data = pd.read_csv('dataset/Invistico_Airline.csv')
data.head()

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0


In [4]:
data.shape

(129880, 22)

In [5]:
# function to check if two decision trees are equal
def are_trees_equal(tree1, tree2):
    # Check that both trees are fitted
    if not hasattr(tree1, 'tree_') or not hasattr(tree2, 'tree_'):
        raise ValueError("Both trees must be fitted before comparison.")

    # Compare parameters
    if tree1.get_params() != tree2.get_params():
        return False

    t1 = tree1.tree_
    t2 = tree2.tree_

    # Compare structure and splitting rules
    attributes_to_check = [
        'children_left', 'children_right',
        'feature', 'threshold',
        'impurity', 'n_node_samples', 'weighted_n_node_samples',
        'value'
    ]

    for attr in attributes_to_check:
        if not np.array_equal(getattr(t1, attr), getattr(t2, attr)):
            return False

    return True


In [14]:
def getConvergenceSampleNum(min_subset_size, max_subset_size, repeat_num, convergence_threshold, sample_size_step=1):
    sample_size_replicablity_dict = {}
    for sample_size in range(min_subset_size, max_subset_size + 1, sample_size_step):
        #get dataset of size sample_size by sampling from the original dataset
        replicable_tree_list = []
        for i in range(repeat_num):
            print(f"sample size: {sample_size}, repeat: {i}")
            X_train, X_test, y_train, y_test = a10.load_dataset(config.dataset_path, sample_size=sample_size, test_size=0.01, random_state=config.random_seed+i)
            H = a10.build_candidate_trees(X_train, y_train, max_depth=config.max_depth, num_trees=config.num_H, random_state=config.random_seed+i)
            tree = a10.replicable_learner(X_train, y_train, H, random_seed=config.random_seed+i)
            replicable_tree_list.append(tree)
            # a = tree.score(X_test, y_test)
        #check the probability if the trees in the replicable_tree_list are the same
        same_tree_count = 0
        for i in range(len(replicable_tree_list)):
            for j in range(i + 1, len(replicable_tree_list)):
                if are_trees_equal(replicable_tree_list[i], replicable_tree_list[j]):
                    same_tree_count += 1
        prob = same_tree_count / (repeat_num * (repeat_num - 1) / 2)
        sample_size_replicablity_dict[sample_size] = prob
    return sample_size_replicablity_dict

            
        
    

In [19]:
ans_dict = getConvergenceSampleNum(min_subset_size=10000, max_subset_size=11000, repeat_num=10, convergence_threshold=0.95, sample_size_step=1000)

sample size: 10000, repeat: 0
Train set size: 10000 Test set size: 1299


100%|██████████| 20/20 [00:00<00:00, 168.05it/s]


k: 74892 v max candidates: 0.07499869129942487 v min candidates: 1.0014246020860022e-06
sample size: 10000, repeat: 1
Train set size: 10000 Test set size: 1299


100%|██████████| 20/20 [00:00<00:00, 209.12it/s]


k: 74892 v max candidates: 0.07499869129942487 v min candidates: 1.0014246020860022e-06
sample size: 10000, repeat: 2
Train set size: 10000 Test set size: 1299


...

k: 74892 v max candidates: 0.07499869129942487 v min candidates: 1.0014246020860022e-06
sample size: 11000, repeat: 8
Train set size: 11000 Test set size: 1299


100%|██████████| 20/20 [00:00<00:00, 154.95it/s]


k: 74892 v max candidates: 0.07499869129942487 v min candidates: 1.0014246020860022e-06
sample size: 11000, repeat: 9
Train set size: 11000 Test set size: 1299


100%|██████████| 20/20 [00:00<00:00, 186.14it/s]


k: 74892 v max candidates: 0.07499869129942487 v min candidates: 1.0014246020860022e-06


In [20]:
ans_dict

{10000: 0.0, 11000: 0.0}